# CIFAR-10 uDNN Model Pre-training + Capuchin

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential, layers
from tensorflow.keras.datasets import cifar10
import matplotlib.pyplot as plt
import random

## Use Google Colab Pro GPU (Optional)

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

## Import CIFAR-10 Dataset

In [ ]:
labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
(trainX, trainY), (testX, testY) = cifar10.load_data()

## Pre-processing Dataset

In [ ]:
trainX = trainX / 255.0
testX = testX / 255.0

## Show an example in dataset (Optional)

In [ ]:
i = random.randint(0, trainX.shape[0])
plt.imshow(trainX[i])
print(labels[trainY[i][0]])

## Define uDNN Model

In [ ]:
model = Sequential([
  layers.Conv2D(filters=16, kernel_size=3, padding="same", activation="relu", input_shape=[32,32,3]),
  layers.Conv2D(filters=16, kernel_size=3, padding="same", activation="relu"),
  layers.MaxPool2D(pool_size=2, strides=2),
  layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu"),
  layers.Conv2D(filters=32, kernel_size=3, padding="same", activation="relu"),
  layers.MaxPool2D(pool_size=2, strides=2),
  layers.Flatten(),
  layers.Dropout(0.5),
  layers.Dense(16),
  layers.Dense(10, activation='softmax')
])
model.summary()

## Compile and Train Model

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='Adam', metrics=['sparse_categorical_accuracy'])

In [ ]:
model.fit(trainX, trainY, epochs=25, batch_size=128, validation_split=0.2)

In [ ]:
model.evaluate(testX, testY)

## Prepare to Export Model

In [ ]:
! git clone https://github.com/leleonardzhang/Capuchin.git
! pip install fxpmath
import sys
from fxpmath import Fxp
sys.path.insert(0, '/content/Capuchin')
import encoder

## Export Model to Header File

In [ ]:
encoder.export_model(model)

## Get Sample Input and Output (Optional)

In [ ]:
test_n = 1                    # the nth data in test dataset
print("Fixed Point Input")
print(Fxp(testX[test_n].transpose(2,0,1), signed = True, n_word = 16, n_frac = 10).val.flatten().tolist())
print("Output Label")
print(testY[test_n])

In [ ]:
plt.imshow(testX[test_n])
print(labels[testY[test_n][0]])

## Next Step

1.   Download header file `neural_network_parameters.h` and copy the header file into the root directory of MSP implementation `capuchin-MCU/`.
2.   Modify the pointer `input_buffer` in `neural_network_parameters.h` and point to desired data input.
3.   Compile and run the program on MSP.
